# The HFM library - A fast marching solver with adaptive stencils 

## Part : Algorithmic enhancements to the fast marching method
## Chapter : Seismic norm models

We illustrate some norm types encountered in seismology, which are symmetric and Finslerian. They are based on various assumptions on the hooke elasticity tensor.

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Sensitivity','FMM'))

In [2]:
from agd import HFMUtils
from agd import AutomaticDifferentiation as ad
from agd import FiniteDifferences as fd
from agd import Metrics
from agd.Metrics import Seismic
from agd.Plotting import savefig; #savefig.dirName = 'Figures/Sensitivity'

In [3]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.optimize

In [4]:
def reload_packages(twice=True):
    from importlib import reload
    HFMUtils = reload(sys.modules['agd.HFMUtils'])
    HFMUtils.reload_submodules()
    fd = reload(sys.modules['agd.FiniteDifferences'])
    Metrics = reload(sys.modules['agd.Metrics'])
    Metrics.reload_submodules()
    ad = reload(sys.modules['agd.AutomaticDifferentiation'])
    ad.reload_submodules()
    global Seismic
    Seismic = reload(sys.modules['agd.Metrics.Seismic'])
    Seismic.reload_submodules()
    if twice: reload_packages(False)

## 1. Reduced norm types

The dual unit ball is defined by an equation of the following form 
$$
	l(X^2,Y^2,Z^2) + q(X^2,Y^2,Z^2) + c X^2 Y^2 Z^2 = 1
$$
where l is linear, q is quadratic, and c is a cubic coefficient, which is relevant only in three dimensions.
The variables $X$, $Y$ and $Z$ denote the coefficients of the input vector, perhaps subject to a linear transformation.

### 1.1 Reproducing Riemann

In [5]:
norm = Seismic.Reduced((1.,2.))
norm.rotate_by(0.5)
norm.rotate_by(-0.5)

In [6]:
norm.gradient(np.array((1,1)))

array([0.81649658, 0.40824829])

In [7]:
norm.norm(np.array((1,1)))

1.224744871391589

### 1.2 Compatibility

In [8]:
delta = ad.Dense.identity(shape=tuple())
norm_ad = Seismic.Reduced( ad.array((1+delta,2.)) )

In [9]:
norm_ad.gradient(np.array([1,1]))

denseAD(array([0.81649658, 0.40824829]),
array([[-0.54433105],
       [ 0.13608276]]))

In [10]:
ad.is_ad(norm_ad._dual_params(),iterables=(tuple,))

agd.AutomaticDifferentiation.Dense.denseAD

In [11]:
norm_ad.norm(np.array([1.,1.]))

denseAD(array(1.22474487),array([-0.40824829]))

In [12]:
np.sqrt(1/(1.+delta)+1/2.)

denseAD(array(1.22474487),array([-0.40824829]))

In [13]:
riem = Metrics.Riemann.from_diagonal(1.+delta,2.).dual()

In [14]:
riem.norm(np.array([1.,1.]))

denseAD(array(1.22474487),array([-0.40824829]))

In [15]:
2*delta

denseAD(array(0.),array([2.]))

In [16]:
val = ad.Dense.denseAD(np.array(0.0),np.array([0.66666667]))

In [17]:
2.*val

denseAD(array(0.),array([1.33333334]))

In [18]:
zdisc = ad.disassociate(np.zeros((2,3,4)),shape_free=(2,3))
zdisc.shape

(2, 3, 1)

In [19]:
mybool = tuple or dict
if mybool:
    print("hu",mybool)

hu <class 'tuple'>


In [20]:
2**-1

0.5

In [21]:
arr=np.ones((2,3))
np.all([arr[1]==1,arr[0]==1])

True

In [22]:
any((True,False))

True

In [23]:
tuple(x for x in norm_ad)

(denseAD(array([1., 2.]),
 array([[1.],
        [0.]])), None, None, None, 6, ())

In [24]:
tuple(norm_ad)

(denseAD(array([1., 2.]),
 array([[1.],
        [0.]])), None, None, None, 6, ())

In [25]:
#hk = Seismic.Hooke.from_Reduced(norm_ad)

## 2. Norms defined by Hooke tensors 

The dual unit ball is as the connected component of the origin, in the set defined by the algebraic equation
$$
    \det(I - m(v)) \geq 0
$$
where $v$ is a co-vector, and $m(v)$ is the symmetric matrix defined by 
$$
    m(v)_{ik} = \sum_{j,l} c_{ijkl} v_j v_l,
$$
where $c$ denotes the Hooke tensor, with shape $d\times d \times d\times d$, and subject to the symmetries $c_{ijkl}=c_{jikl}=c_{klij}$.

The Hooke tensor is internally stored in Voigt notation, as a symmetric matrix of shape $d' \times d'$ where $d'=d(d+1)/2$. One typically assumes that $c$ is positive definite, an establishes under this assumption that the dual unit ball is uniformly convex, so that a norm is indeed defined.

### 2.1 Reproducing Riemann

In [26]:
riem = Metrics.Riemann.from_diagonal(1.,2.)
norm = Seismic.Hooke.from_cast(riem)

In [27]:
riem.m

array([[1., 0.],
       [0., 2.]])

In [28]:
riem = Metrics.Riemann.from_diagonal((1.,2.))

In [29]:
ad.array((1,2))

array([1, 2])

In [30]:
norm.hooke

array([[1.        , 0.70710678, 0.        ],
       [0.70710678, 0.5       , 0.        ],
       [0.        , 0.        , 0.        ]])

In [31]:
v=np.array((0.,1.))
norm.norm(v)

1.414213562373095

In [32]:
norm.gradient(v)

array([0.        , 1.41421356])

In [34]:
red = Seismic.Reduced.from_Hooke(norm)

In [35]:
red.niter_sqp

6

In [36]:
red.linear

array([1. , 0.5])

In [37]:
if tuple():
    print("hi")